In [2]:
import pandas as pd
from datetime import date, timedelta

In [3]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [4]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [5]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [6]:
f = open('call_rate.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', 'price', 'label']
['2024-03-21\t3.5290\t-0.11\n', '2024-03-20\t3.5330\t0.56\n', '2024-03-19\t3.5130\t2.09\n', '2024-03-18\t3.4410\t-0.66\n', '2024-03-15\t3.4640\t-0.63\n']


In [7]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None,
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', 'price', 'label'])

    # '날짜' 열을 datetime 형식으로 변환\n ,
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인\n ,
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['price'] > prev_row['price']:
            label = '1'
        elif row['price'] == prev_row['price']:
            label = 0
        else:
            label = '-1'

        df_ud.loc[i] = [time.date(), row['price'], label]

    return df_ud 

In [8]:
df = pd.DataFrame(new_datas)
df
result = label_call(df)
result

,date,price,label
0,2024-03-21,3.5290,-1
1,2024-03-20,3.5330,-1
2,2024-03-19,3.5130,-1
3,2024-03-18,3.4410,-1
4,2024-03-15,3.4640,-1
...,...,...,...
3253,2011-01-07,2.4900,0
3254,2011-01-06,2.4900,0
3255,2011-01-05,2.4900,0
3256,2011-01-04,2.5000,1


In [9]:
pip install ekonlpy

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
import pandas as pd
import numpy as np
import re
from pandas import Series, DataFrame 

In [11]:
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

# 패턴 정의\n ,
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/edaily_news_test.json'

df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))

news1_df = df

news1_df['words']=" "  #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = news1_df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가\n ,

    except Exception as ex :
        print("fail")
        
news1_df = news1_df.drop(['content', 'title'], axis=1)  # axis=1은 열을 의미함
news1_df.columns
# df.to_csv(\ news_test.csv\ , mode='w') 

Index(['date', 'words'], dtype='object')

In [12]:
news1_df

,date,words
298,2023-05-04,fed/NNG.fed/NNG.금리/NNG.포인트/NNG.인상/NNG.하/XSV.가운...
288,2023-05-04,fed/NNG.fed/NNG.금리/NNG.인상/NNG.종료/NNG.시사/NNG.하/...
297,2023-05-04,fed/NNG.fed/NNG.매파/NNG.색채/NNG.드러나/VV.fed/NNG.예...
296,2023-05-04,fed/NNG.fed/NNG.fed/NNG.금리/NNG.수준/NNG.인상/NNG.하...
287,2023-05-04,fed/NNG.fed/NNG.현지/NNG.개최/NNG.달/NNG.fed/NNG.fe...
...,...,...
216,2012-09-13,영세자영업자/NNG.대해/VV.규모/NNG.자금/NNG.공급/NNG.되/VV.고금리...
215,2012-09-13,금리/NNG.수준/NNG.동결/NNG.하/VV.경기/NNG.위축/NNG.되/XSV....
212,2012-09-12,현지/NNG.뉴욕/NNG.달러/NNG.규모/NNG.만기/NNG.글로벌본드/NNG.성...
211,2012-09-11,대박/NNG.나/VV.금리/NNG.더/MAG.얹/VV.적금/NNG.상품/NNG.인기...


In [13]:
mpck = MPCK()

# 패턴 정의\n ,
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/joongamg_news_test.json'

df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))

news2_df = df

news2_df['words']=" "  #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = news2_df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")

news2_df = news2_df.drop(['content', 'title'], axis=1)  # axis=1은 열을 의미함
news2_df.columns
# df.to_csv(\ news_test.csv\ , mode='w') 

Index(['date', 'words'], dtype='object')

In [14]:
news2_df

,date,words
4,2024-03-21,업/NNG.비트/NNG.라운지/NNG.비트코인/NNG.시세/NNG.나오/VV.뉴스/...
5,2024-03-21,마이너스금리/NNG.정책/NNG.종료/NNG.선언/NNG.추가/NNG.금리/NNG....
6,2024-03-21,최저/NNG.금리/NNG.주택구입자금/NNG.빌리/VV.신생아/NNG.특례/NNG....
0,2024-03-15,가명/NNG.아이/NNG.낳/VV.신생아/NNG.특례/NNG.대출/NNG.받/VV....
2,2024-03-15,공장/NNG.근무/NNG.하/XSV.근로자/NNG.생산자/NNG.물가/NNG.소비자...
...,...,...
69,2023-12-29,左/NNG.右/NNG.이끄/VV.고물가/NNG.수출/NNG.부진/NNG.성장률/NN...
66,2023-12-29,과/NNG.논설위원/NNG.총선/NNG.다가오/VV.변화/NNG.바람/NNG.불어오...
65,2023-12-29,금융안정/NNG.열/VV.금융안정보고서/NNG.설명회/NNG.가계/NNG.대출/NN...
63,2023-12-29,개인사업자/NNG.대출/NNG.받/VV.소상공인/NNG.자영업자/NNG.규모/NNG...


In [15]:
mpck = MPCK()

# 패턴 정의
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/moneytoday_news_test.json'

df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['content'] = df['content'].apply(lambda x: re.sub(pattern, '', x))

news3_df = df

news3_df['words']=" " #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = news3_df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")

news3_df = news3_df.drop(['content', 'title'], axis=1)  # axis=1은 열을 의미함
news3_df.columns
# df.to_csv(\"news_test.csv\", mode='w')

Index(['date', 'words'], dtype='object')

In [16]:
news3_df

,date,words
28,2019-08-04,수출/NNG.규제/NNG.따르/VV.피해기업/NNG.임직원/NNG.지원/NNG.하/...
30,2019-08-04,수출/NNG.규제/NNG.피해/NNG.보/VV.중소기업/NNG.대해/VV.최대/NN...
31,2019-08-03,수출/NNG.규제/NNG.피해기업/NNG.대해/VV.신규/NNG.자금/NNG.지원/...
26,2019-08-03,시/NNG.주민/NNG.반발/NNG.곤욕/NNG.치렀/VV.시설/NNG.설치/NNG...
27,2019-08-02,기념/NNG.특별/NNG.금리/NNG.이벤트/NNG.진행/NNG.하/VV.밝히/VV...
...,...,...
1,2012-10-11,투자/NNG.증권/NNG.장/NNG.금리/NNG.인하/NNG.대해/VV.증시/NNG...
7,2012-10-11,예상/NNG.금리/NNG.인하/NNG.하/VV.달/NNG.다/MAG.경기/NNG.부...
3,2012-10-11,금리/NNG.연/VV.인하/NNG.하/VV.금리/NNG.떨어/VV.대해/VV.금리/...
2,2012-10-11,금리/NNG.결정/NNG.앞두/VV.금리/NNG.인하/NNG.기대/NNG.동반/NN...


In [17]:
mpck = MPCK()

# 패턴 정의\n",
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/minutes_re.tsv'

df = pd.read_csv(result_path, sep='\t')

# 특정패턴 제거
for pattern in news_pattern:
    df['내용'] = df['내용'].apply(lambda x: re.sub(pattern, '', x))

min_df = df

min_df['words']=" " #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = min_df['내용']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")

min_df = min_df.drop('내용', axis=1)  # axis=1은 열을 의미함
min_df.columns
# df.to_csv(\"news_test.csv\", mode='w')

Index(['날짜', 'words'], dtype='object')

In [18]:
min_df = min_df.rename(columns={'날짜': 'date'})
min_df

,date,words
0,2006-05-11,조/NNG.의거/NNG.하/XSV.의안/NNG.통화정책/NNG.방향/NNG.상정/N...
1,2006-05-25,담당/NNG.보가/NNG.지급/NNG.결제/NNG.원활/NNG.위하/VV.금융기관/...
2,2006-06-08,조/NNG.의거/NNG.하/XSV.의안/NNG.통화정책/NNG.방향/NNG.상정/N...
3,2006-06-22,조건/NNG.구비/NNG.하/XSV.사항/NNG.명시/NNG.하/XSV.차액/NNG...
4,2006-07-07,보/VV.금융시장/NNG.동향/NNG.대하/VV.보고/NNG.내용/NNG.별첨/NN...
...,...,...
308,2023-04-11,토의/NNG.내용/NNG.개최/NNG.되/VV.동향/NNG.같/VA.토의/NNG.있...
309,2024-01-11,토의/NNG.내용/NNG.개최/NNG.되/VV.동향/NNG.같/VA.토의/NNG.있...
310,2024-01-25,제안/NNG.설명/NNG.이/VV.협의회/NNG.논의/NNG.되/VV.내용/NNG....
311,2024-02-08,토의/NNG.내용/NNG.제안/NNG.설명/NNG.이/VV.발권/NNG.규정/NNG...


In [19]:
mpck = MPCK()

# 패턴 정의
news_pattern = ['\\[.*\\]','(?<=[가-힣])\\.','\\w{4,}\\@[a-zA-Z0-9\\-]{2,}\\.[a-z]{2,}(\\.[a-z]{2})?','....기자']
bone_pattern = []

result_path = './crawl_result/text_by_date.json1_test.json'

df = pd.read_json(result_path)
df = df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

# 특정패턴 제거
for pattern in news_pattern:
    df['text'] = df['text'].apply(lambda x: re.sub(pattern, '', x))

bond_df = df

bond_df['words']=" " #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = bond_df['text']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

    except Exception as ex :
        print("fail")

bond_df = bond_df.drop('text', axis=1)  # axis=1은 열을 의미함
bond_df.columns
# df.to_csv(\"news_test.csv\", mode='w')


Index(['date', 'words'], dtype='object')

In [20]:
bond_df

,date,words
190,2011-04-11,공사채/NNG.발행/NNG.내역/NNG.단위/NNG.종목명/NNG.표면금리/NNG....
189,2011-04-11,공사채/NNG.발행/NNG.내역/NNG.단위/NNG.종목명/NNG.표면금리/NNG....
188,2011-04-11,공사채/NNG.발행/NNG.내역/NNG.단위/NNG.종목명/NNG.표면금리/NNG....
187,2011-04-11,시장/NNG.요약/NNG.주/NNG.회사채/NNG.거래/NNG.주/NNG.신용등급/...
186,2011-04-11,시장/NNG.요약/NNG.주/NNG.회사채/NNG.거래/NNG.주/NNG.신용등급/...
...,...,...
4,2011-01-07,금리/NNG.국고채/NNG.금리/NNG.추이/NNG.일드커브/NNG.형태/NNG.변...
3,2011-01-04,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
2,2011-01-02,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
1,2011-01-01,에너지기업/NNG.유가/NNG.급락/NNG.트레이드/NNG.증권/NNG.본부/NNG...


In [21]:
# concat을 사용하여 데이터프레임 합치기
news = pd.concat([news1_df, news2_df, news3_df, min_df, bond_df])
news['date'] = pd.to_datetime(news['date']).dt.date

news

,date,words
298,2023-05-04,fed/NNG.fed/NNG.금리/NNG.포인트/NNG.인상/NNG.하/XSV.가운...
288,2023-05-04,fed/NNG.fed/NNG.금리/NNG.인상/NNG.종료/NNG.시사/NNG.하/...
297,2023-05-04,fed/NNG.fed/NNG.매파/NNG.색채/NNG.드러나/VV.fed/NNG.예...
296,2023-05-04,fed/NNG.fed/NNG.fed/NNG.금리/NNG.수준/NNG.인상/NNG.하...
287,2023-05-04,fed/NNG.fed/NNG.현지/NNG.개최/NNG.달/NNG.fed/NNG.fe...
...,...,...
4,2011-01-07,금리/NNG.국고채/NNG.금리/NNG.추이/NNG.일드커브/NNG.형태/NNG.변...
3,2011-01-04,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
2,2011-01-02,거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
1,2011-01-01,에너지기업/NNG.유가/NNG.급락/NNG.트레이드/NNG.증권/NNG.본부/NNG...


In [22]:
news['words']

298    fed/NNG.fed/NNG.금리/NNG.포인트/NNG.인상/NNG.하/XSV.가운...
288    fed/NNG.fed/NNG.금리/NNG.인상/NNG.종료/NNG.시사/NNG.하/...
297    fed/NNG.fed/NNG.매파/NNG.색채/NNG.드러나/VV.fed/NNG.예...
296    fed/NNG.fed/NNG.fed/NNG.금리/NNG.수준/NNG.인상/NNG.하...
287    fed/NNG.fed/NNG.현지/NNG.개최/NNG.달/NNG.fed/NNG.fe...
                             ...                        
4      금리/NNG.국고채/NNG.금리/NNG.추이/NNG.일드커브/NNG.형태/NNG.변...
3      거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
2      거래주체/NNG.매매동향/NNG.은행/NNG.순매수/NNG.상위/NNG.은행/NNG...
1      에너지기업/NNG.유가/NNG.급락/NNG.트레이드/NNG.증권/NNG.본부/NNG...
0      회사채/NNG.등급별/NNG.분포/NNG.현황/NNG.단위/NNG.개/NNG.구분/...
Name: words, Length: 1401, dtype: object

In [23]:
# # korean stop words 불러오기
# stopwords_path = 'korean_sotp_words.txt'
# stopwords = []
# with open(stopwords_path, 'r', encoding='utf-8') as file:
#     for line in file:
#         stopwords.append(line.strip())
# print(stopwords)  # 불용어 모음

In [24]:
# word = []
# for tag in text: #토큰화된 text
#     if tag[1] not in stopwords:
#         if tag[0] not in stopwords:
#             word.append(tag[0])

In [26]:
import re

# 정규표현식을 사용하여 한글 단어와 쉼표를 추출하는 함수
def extract_hangul_words(text):
    words = ' '.join(re.findall('[가-힣]+', text))  # 쉼표와 함께 한글 단어를 문자열로 반환
    return words

# 시리즈의 각 항목에 함수를 적용하여 한글 단어와 쉼표를 추출
news['words'] = news['words'].apply(extract_hangul_words)

In [27]:
news

,date,words
298,2023-05-04,금리 포인트 인상 하 가운데 시장 금리 하락 보 전망 나오 금리 스프레드 금리갭 높...
288,2023-05-04,금리 인상 종료 시사 하 현지 개최 달 통하 금리 포인트 인상 내 통하 통화정책 누...
297,2023-05-04,매파 색채 드러나 예상 포인트 금리 오르 추가 인상 없 신호 통하 보내 확언 하 않...
296,2023-05-04,금리 수준 인상 하 한미 금리 역전폭 포인트 벌어지 역대 최대 수준 다 금리 인상 ...
287,2023-05-04,현지 개최 달 통하 금리 포인트 인상 하 들 베이비스텝 단행 하 금리 높 금리 갭 벌어지
...,...,...
4,2011-01-07,금리 국고채 금리 추이 일드커브 형태 변화 국고채 통안채 통안채 국고채 국고채 국고...
3,2011-01-04,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
2,2011-01-02,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
1,2011-01-01,에너지기업 유가 급락 트레이드 증권 본부 국제 유가 바닥 잡 지지부진 흐름 보 기업...


In [28]:
import pandas as pd

# Date 열을 datetime으로 변환하고 인덱스로 설정
result['date'] = pd.to_datetime(result['date'])
result.set_index('date', inplace=True)

news['date'] = pd.to_datetime(news['date'])
news.set_index('date', inplace=True)

# 중복된 인덱스 제거
result = result[~result.index.duplicated()]
news = news[~news.index.duplicated()]

# 동일한 날짜에 대해 열을 합치기
merged_df = pd.concat([result, news], axis=1)

# 'date' 열을 다시 열로 변환
merged_df.reset_index(inplace=True)

# 결과 출력
print(merged_df.columns)
print(merged_df)

Index(['date', 'price', 'label', 'words'], dtype='object')
           date   price label  \
0    2006-05-11     NaN   NaN   
1    2006-05-25     NaN   NaN   
2    2006-06-08     NaN   NaN   
3    2006-06-22     NaN   NaN   
4    2006-07-07     NaN   NaN   
...         ...     ...   ...   
3401 2024-03-15  3.4640    -1   
3402 2024-03-18  3.4410    -1   
3403 2024-03-19  3.5130    -1   
3404 2024-03-20  3.5330    -1   
3405 2024-03-21  3.5290    -1   

                                                  words  
0     조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 국제 유가 ...  
1     담당 보가 지급 결제 원활 위하 금융기관 일중 당좌대출 한도규제 폐지 하 일중 당좌...  
2     조 의거 하 의안 통화정책 방향 상정 하 토의 내용 국내외 경제동향 하 경기 선행 ...  
3     조건 구비 하 사항 명시 하 차액 결제 참가 기관 자금 조정 적수 보전 이자 정산 ...  
4     보 금융시장 동향 대하 보고 내용 별첨 조 의거 하 의안 통화정책 방향 상정 하 토...  
...                                                 ...  
3401  가명 아이 낳 신생아 특례 대출 받 대기업 근무 하 같 남편 소득 합하 달하 신생아...  
3402                                                NaN  
3403                  

In [29]:
# 'label' 칼럼의 값이 '매파'인 행과 '비둘기파'인 행으로 데이터프레임 분리
df_A = merged_df[merged_df['label'] == '1']
df_B = merged_df[merged_df['label'] == '-1']

# Nan인 행 제거
df_A = df_A.dropna()
df_B = df_B.dropna()

# 필요없는 열 제거 
df_A = df_A.drop(['date', 'price'], axis=1)
df_B = df_B.drop(['date', 'price'], axis=1)

word_df = pd.concat([df_A,df_B],axis=0)
x_tain = word_df["words"]
y_tain = word_df["label"]

In [30]:
df_A = df_A.rename(columns={'label': '매파'})
df_A
# df_A.to_csv("hawkish_test_2.csv", mode='w')

,매파,words
107,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
111,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
113,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
114,1,조 의거 하 의안 호 통화정책 방향 상정 하 보 내용 접수 하 토의 내용 국내외 경...
115,1,표 회사채 만기 종목 금액 순 종목명 발행일 만기일 만기금액 표면금리 신용등급 물산...
...,...,...
3372,1,미 현지 열 회견 질문 답 하 숨겨둔 발톱 다시 드러나 긴축 완화 적절 금리 인하 ...
3375,1,반전 없 현지 치러진 공식 대선 경선 예상 압도 다수 지지 오르 공화당 대선 굳어진...
3376,1,금리 인하 조심 접근 하 미 여파 현지 미 국채 금리 급등 보 트레이드 웹 따르 미...
3378,1,토의 내용 제안 설명 이 발권 규정 개정 내용 함께 협의회 논의 되 내용 같이 보 ...


In [31]:
len(df_A['words'])

171

In [32]:
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter

# 텍스트 칼럼에서 단어 횟수 세기
text = ' '.join(df_A['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# 빈도수가 15 이상인 단어 필터링
filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

# 데이터프레임 생성
df_A_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])

df_A_counts
# df_A_counts.to_csv("hawkish_count_1.csv", mode='w')

,Word,Frequency
0,은행,843
1,순매수,153
2,상위,93
3,순매도,132
4,보험기금,22
...,...,...
1698,병목,30
1699,오미크론,27
1700,우크라이나사태,77
1701,빈일자리,26


In [53]:
df_A_counts['P'] = 1

In [40]:

df_A_counts = df_A_counts.drop('매파', axis=1)

In [54]:
df_A_counts

,Word,Frequency,P
0,은행,843,1
1,순매수,153,1
2,상위,93,1
3,순매도,132,1
4,보험기금,22,1
...,...,...,...
1698,병목,30,1
1699,오미크론,27,1
1700,우크라이나사태,77,1
1701,빈일자리,26,1


In [55]:

df_A_counts.to_csv("hawkish_count_1.csv", mode='w')

In [42]:
# 텍스트 칼럼에서 단어 횟수 세기
text = ' '.join(df_B['words'])  # 텍스트 칼럼의 모든 텍스트를 하나의 문자열로 결합
tokens = word_tokenize(text)  # 텍스트를 단어로 토큰화
word_counts = Counter(tokens)  # 단어 등장 횟수 카운트

# 빈도수가 15 이상인 단어 필터링
filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 15}

# 데이터프레임 생성
df_B_counts = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])

df_B_counts
# df_B_counts.to_csv("dovish_count_1.csv", mode='w')

,Word,Frequency
0,표,37
1,만기,157
2,회사채,440
3,종목,24
4,신용등급,94
...,...,...
1939,대면,21
1940,변이,17
1941,자녀,24
1942,전기차,37


In [45]:
df_B_counts['N'] = 1

In [49]:
df_B_counts = df_B_counts.drop('P', axis=1)

In [50]:
df_B_counts

,Word,Frequency,N
0,표,37,1
1,만기,157,1
2,회사채,440,1
3,종목,24,1
4,신용등급,94,1
...,...,...,...
1939,대면,21,1
1940,변이,17,1
1941,자녀,24,1
1942,전기차,37,1


In [56]:
df_B_counts.to_csv("dovish_count_1.csv", mode='w')

In [ ]:

   "source": [
    "from sklearn.feature_extraction.text import CountVectorizer\n",
    "from sklearn.naive_bayes import MultinomialNB\n",
    "from ekonlpy import Mecab\n",
    "from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n",
    "from sklearn.feature_extraction.text import TfidfVectorizer\n",
    "\n",
    "# TfidfVectorizer 설정\n",
    "tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.0)  # max_df 값을 1.0으로 설정하여 모든 단어를 포함하도록 함\n",
    "\n",
    "# 문서를 벡터화\n",
    "tfidf_matrix = tfidf_vectorizer.fit_transform(x_tain)\n",
    "\n",
    "# accuracy_score(y_test, y_pred)\n",
    "# precision_score(y_test, y_pred)\n",
    "# recall_score(y_test, y_pred)\n",
    "# f1_score(y_test, y_pred)\n",
    "\n",
    "#모델 훈련및 결과출력\n",
    "# def Model_test(word_list,file_text):\n",
    "    # 1. 데이터 준비\n",
    "X_train = x_tain\n",
    "y_train = y_tain\n",
    "print(\"학습물을 다 읽었습니다\")\n",
    "# 2. 특성 추출 (형태소 분석 및 n-그램)\n",
    "def taging(text):\n",
    "    return text.split()\n",
    "#tagger = taging(text)\n",
    "mecab_tokenize = lambda text: taging(text)\n",
    "vectorizer = CountVectorizer(tokenizer=mecab_tokenize, ngram_range=(1, 5), min_df=15)\n",
    "X_train_counts = vectorizer.fit_transform(X_train)\n",
    "print(\"벡터라이즈를 생성하였습니다\")\n",
    "print(\"학습을 시작합니다\")\n",
    "# 3. 나이브 베이즈 모델 학습\n",
    "clf = MultinomialNB()\n",
    "clf.fit(X_train_counts, y_train)\n",
    "print(\"학습을 시작합니다\")\n",
    "print(clf.score(X_train_counts, y_train))\n",
    "#4. 새로운 입력 예측\n",
    "X_test = [file_text]\n",
    "X_test_counts = vectorizer.transform(X_test)\n",
    "predicted_labels = clf.predict(X_test_counts)\n",
    "print(predicted_labels)"
   ]

In [71]:
word_df

,label,words
107,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
111,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
113,1,거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금...
114,1,조 의거 하 의안 호 통화정책 방향 상정 하 보 내용 접수 하 토의 내용 국내외 경...
115,1,표 회사채 만기 종목 금액 순 종목명 발행일 만기일 만기금액 표면금리 신용등급 물산...
...,...,...
3366,-1,제안 설명 이 협의회 논의 되 내용 같이 보 하 공개시장 운영 대상 기관 확대 취지...
3368,-1,소비자 장 보 뉴스 세계 인플레이션 주춤 하 긴축 통화정책 피벗 통화정책 전환 기대...
3369,-1,구미 축제 인파 몰리 북새통 이루 자치 단체 개최 축제 매출 상승 크 기여 하 분석...
3401,-1,가명 아이 낳 신생아 특례 대출 받 대기업 근무 하 같 남편 소득 합하 달하 신생아...


In [72]:
import nltk
from nltk import bigrams, word_tokenize
from nltk.util import ngrams

nltk.download('punkt')

sentence = "거래주체 매매동향 은행 순매수 상위 은행 순매도 상위 보험기금 순매수 상위 보험기금"
tokens = word_tokenize(sentence)

bigram = list(bigrams(tokens))
trigram = list(ngrams(tokens, 3))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SDA11\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [73]:
print("Bigrams:", bigram)
print("Trigrams:", trigram)

Bigrams: [('거래주체', '매매동향'), ('매매동향', '은행'), ('은행', '순매수'), ('순매수', '상위'), ('상위', '은행'), ('은행', '순매도'), ('순매도', '상위'), ('상위', '보험기금'), ('보험기금', '순매수'), ('순매수', '상위'), ('상위', '보험기금')]
Trigrams: [('거래주체', '매매동향', '은행'), ('매매동향', '은행', '순매수'), ('은행', '순매수', '상위'), ('순매수', '상위', '은행'), ('상위', '은행', '순매도'), ('은행', '순매도', '상위'), ('순매도', '상위', '보험기금'), ('상위', '보험기금', '순매수'), ('보험기금', '순매수', '상위'), ('순매수', '상위', '보험기금')]


In [ ]:
# def generate_ngrams(s, n):
#     tokens = s.split()
#     ngrams = zip(*[tokens[i:] for i in range(n)])
    
#     return ["_".join(ngram) for ngram in ngrams]

In [ ]:
# word_df['1_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 1))))
# print("1 그램 생성")
# word_df['2_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 2))))
# print("2 그램 생성")
# word_df['3_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 3))))
# print("3 그램 생성")
# word_df['4_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 4))))
# print("4 그램 생성")
# word_df['5_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 5))))
# print("5 그램 생성")
# word_df.to_csv('data_tokens_ngram_1_5.csv', index=False)